In [50]:
import os
import cv2
import numpy as np
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
import time

In [51]:
people = ["Alejandra Garcia", "Mauricio Vargas", "Sebastian Rojas", "Ignacio Valencia"]

def create_label_dict(names, size):
    label_dict = {}
    for name in names:
        label = np.zeros(size)
        label[names.index(name)] = 1
        label_dict[name] = label
    return label_dict

size_people = len(people)
name_pos_dict = create_label_dict(people, size_people)

def my_data():
    data = []
    for img in tqdm(os.listdir("images")):
        path = os.path.join("images", img)
        img_name = img.split('.')[0]
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        try:
            img_data = cv2.resize(img_data, (50, 50))
        except Exception as e:
            print(e)
        if img_name in name_pos_dict:
            data.append([np.array(img_data), name_pos_dict[img_name]])
    shuffle(data)
    return data

data = my_data()
len(data)

train_percentage = 0.17
split_point = int(len(data) * train_percentage)

train = data[:split_point]
test = data[split_point:]
X_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
print(X_train.shape)
y_train = np.array([i[1] for i in train]) 
X_test = np.array([i[0] for i in test]).reshape(-1, 50, 50, 1)
print(X_test.shape)
y_test = np.array([i[1] for i in test]) 


100%|██████████| 397/397 [00:00<00:00, 2761.52it/s]

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

(67, 50, 50, 1)
(329, 50, 50, 1)


In [52]:
model_name = "model_upb_3_{}".format(int(time.time()))

log_dir = "tensor-logs/{}".format(model_name) 

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(50, 50, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=(5, 5)))
model.add(keras.layers.Conv2D(128, (5, 5), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(5, 5)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(size_people, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=log_dir)

model.fit(X_train, y_train, epochs=21, validation_data=(X_test, y_test), verbose=1, callbacks=[tensorboard_callback])

model.save("model_upb_3.h5")

Epoch 1/21
3/3 [==============================] - 2s 170ms/step - loss: 22.5791 - accuracy: 0.2388 - val_loss: 30.0127 - val_accuracy: 0.2340
Epoch 2/21
3/3 [==============================] - 0s 67ms/step - loss: 32.5366 - accuracy: 0.2537 - val_loss: 7.1470 - val_accuracy: 0.2492
Epoch 3/21
3/3 [==============================] - 0s 64ms/step - loss: 8.4221 - accuracy: 0.2537 - val_loss: 6.3010 - val_accuracy: 0.2462
Epoch 4/21
3/3 [==============================] - 0s 68ms/step - loss: 6.7388 - accuracy: 0.1940 - val_loss: 3.7951 - val_accuracy: 0.2523
Epoch 5/21
3/3 [==============================] - 0s 67ms/step - loss: 3.0266 - accuracy: 0.2388 - val_loss: 2.6564 - val_accuracy: 0.2492
Epoch 6/21
3/3 [==============================] - 0s 68ms/step - loss: 2.8242 - accuracy: 0.2687 - val_loss: 1.6802 - val_accuracy: 0.2553
Epoch 7/21
3/3 [==============================] - 0s 66ms/step - loss: 1.7052 - accuracy: 0.3731 - val_loss: 1.5820 - val_accuracy: 0.2462
Epoch 8/21
3/3 [=======